# Video Generation Pipeline - Google Colab

This notebook handles heavy video rendering tasks using Colab's free GPU/CPU resources.

## Setup Instructions:
1. ✅ Set your API URL (Cell 16)
2. ✅ Upload your project ZIP file (Cell 6)
3. ✅ Install dependencies (Cells 2-4, 9)
4. ✅ Configure environment (Cell 14)
5. ✅ Run rendering (Cell 17) - automatically processes jobs from your API
6. ✅ Download output video (Cell 19)

## API Mode
This notebook uses **API mode** - jobs are created via your API server and automatically processed here. No manual JSON upload needed!


## 1. Install Dependencies


In [ ]:
# Install Node.js (v20)
!curl -fsSL https://deb.nodesource.com/setup_20.x | bash -
!apt-get install -y nodejs

# Verify installation
!node --version
!npm --version


In [ ]:
# Install FFmpeg (required for Remotion)
!apt-get update
!apt-get install -y ffmpeg

# Verify FFmpeg
!ffmpeg -version | head -n 1


In [ ]:
# Install Chrome/Chromium for Remotion
!apt-get install -y chromium-browser chromium-chromedriver

# Set Chrome path for Remotion
import os
os.environ['REMOTION_BROWSER_EXECUTABLE'] = '/usr/bin/chromium-browser'


## 2. Upload Project Files


In [ ]:
from google.colab import files
import zipfile
import os

# Create project directory
!mkdir -p /content/video-gen

print("📦 Upload your project ZIP file (colab-project.zip):")
print("   Option 1: Upload ZIP file (recommended)")
print("   Option 2: Use git clone if your project is in a repository")
print("")

# Option 1: Upload ZIP file
uploaded = files.upload()

# Extract ZIP if uploaded
for filename in uploaded.keys():
    if filename.endswith('.zip'):
        print(f"📂 Extracting {filename}...")
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall('/content/video-gen')
        print(f"✅ Extracted {filename} to /content/video-gen")
        os.remove(filename)  # Clean up

# Option 2: Git clone (uncomment if using)
# !git clone YOUR_REPO_URL /content/video-gen

# Change to project directory
os.chdir('/content/video-gen')
print(f"✅ Current directory: {os.getcwd()}")
print(f"📁 Project files: {', '.join(os.listdir('.'))}")


In [ ]:
# Verify we're in the project directory
import os
if os.getcwd() != '/content/video-gen':
    os.chdir('/content/video-gen')

print(f"✅ Current directory: {os.getcwd()}")
print(f"📁 Project files: {', '.join(os.listdir('.'))}")


## 3. Install Project Dependencies


In [ ]:
# Install npm dependencies
!npm install

print("Dependencies installed!")


## 4. Upload Video Data (JSON Plan)

Skipped. This notebook runs in API mode and does not require manual JSON uploads.


In [ ]:
# Skipped: Manual JSON upload is not needed in API mode.
json_file = None
print("ℹ️  JSON upload skipped. Jobs are fetched from your API.")


In [ ]:
# Upload assets if needed (images, audio files, etc.)
print("Upload assets folder (optional - if your plan references local assets):")
# assets_uploaded = files.upload()
# You can also use a ZIP file for assets

# Create assets directory structure
!mkdir -p public/assets/gemini-images
!mkdir -p public/assets/voiceovers
!mkdir -p public/assets/vectorized


## 5. Configure Environment


In [ ]:
# Set environment variables for Colab
import os

# Remotion configuration
os.environ['REMOTION_BROWSER_EXECUTABLE'] = '/usr/bin/chromium-browser'
os.environ['REMOTION_BROWSER_TIMEOUT'] = '120000'  # 2 minutes

# FFmpeg configuration
os.environ['FFMPEG_BINARY'] = '/usr/bin/ffmpeg'
os.environ['FFPROBE_BINARY'] = '/usr/bin/ffprobe'

# Optional: Set API keys if needed
# os.environ['GEMINI_API_KEY'] = 'your-api-key'
# os.environ['DEEPGRAM_API_KEY'] = 'your-api-key'

print("Environment configured!")


## 6. Render Video


In [ ]:
# Set your API server URL (REQUIRED for API mode)
API_BASE_URL = "https://iesha-ordainable-cullen.ngrok-free.dev"  # Your ngrok URL

print(f"📡 API URL: {API_BASE_URL}")
print("✅ Ready for API mode - jobs will be fetched automatically")
print("   No need to create render script manually - the next cell handles it!")


In [ ]:
# Video Rendering - API Mode (Automatic Job Processing)
import requests
import json
import os
import subprocess

# Preflight: ensure we are in project root and deps exist
try:
    cwd = os.getcwd()
    if cwd != '/content/video-gen':
        os.chdir('/content/video-gen')
        print(f"📁 Changed directory to: {os.getcwd()}")
except Exception as e:
    print(f"⚠️ Could not change directory: {e}")

# Install deps if missing
try:
    need_install = not os.path.exists('node_modules')
    if need_install:
        print('📦 Installing npm deps (first run)...')
        print(subprocess.check_output(['npm','install'], text=True))
    # Ensure ts-node/typescript/@types/node and axios are present
    def ensure_pkg(pkg, dev=False):
        pkg_json = 'package.json'
        try:
            with open(pkg_json,'r') as f:
                p = json.load(f)
            present = (pkg in p.get('dependencies',{})) or (pkg in p.get('devDependencies',{}))
        except Exception:
            present = False
        if not present:
            print(f"➕ Installing {pkg}...")
            args = ['npm','i', pkg]
            if dev:
                args.insert(2,'-D')
            print(subprocess.check_output(args, text=True))
    ensure_pkg('ts-node', dev=True)
    ensure_pkg('typescript', dev=True)
    ensure_pkg('@types/node', dev=True)
    ensure_pkg('axios', dev=False)
except subprocess.CalledProcessError as e:
    print('⚠️ Dependency install output:')
    print(e.output)
except Exception as e:
    print(f'⚠️ Dependency setup warning: {e}')

# Verify renderer exists
renderer_path = 'server/services/remotion-ai-renderer.ts'
if not os.path.exists(renderer_path):
    print('❌ Renderer not found at server/services/remotion-ai-renderer.ts')
    print('👉 Re-upload colab-project.zip (from your repo) in Cell 6, then run Cells 7 and 9.')

# Set API URL - use default ngrok URL
api_url = 'https://iesha-ordainable-cullen.ngrok-free.dev'

# Try to get from globals if Cell 16 was run
if 'API_BASE_URL' in globals() and globals().get('API_BASE_URL'):
    api_url = globals()['API_BASE_URL']
    print(f"✅ Using API_BASE_URL from Cell 16: {api_url}")
else:
    print(f"📡 Using default ngrok URL: {api_url}")

print(f"🔍 Starting API mode...")
print(f"   API URL: {api_url}")
print("")

# Poll for pending jobs from your API server
# Note: ngrok free tier shows a warning page - headers help bypass it
try:
    # Add headers to bypass ngrok warning page
    headers = {
        'User-Agent': 'Mozilla/5.0 (compatible; ColabBot/1.0)',
        'Accept': 'application/json',
    }
    response = requests.get(f"{api_url}/api/colab/jobs/pending", timeout=10, headers=headers, allow_redirects=True)
    
    if response.status_code == 200:
        pending_jobs = response.json().get('jobs', [])
        print(f"✅ Found {len(pending_jobs)} pending jobs")
        
        if len(pending_jobs) == 0:
            print("ℹ️  No pending jobs. Create a job via API to process it here.")
        else:
            for job in pending_jobs:
                job_id = job['jobId']
                plan_url = job['planUrl']
                callback_url = f"{api_url}/api/colab/callback/{job_id}"
                
                print(f"\n🎬 Processing job {job_id}...")
                
                # Download plan
                if plan_url.startswith('/'):
                    full_plan_url = f"{api_url}{plan_url}"
                else:
                    full_plan_url = plan_url
                
                print(f"📥 Downloading plan from: {full_plan_url}")
                plan_headers = {
                    'User-Agent': 'Mozilla/5.0 (compatible; ColabBot/1.0)',
                    'Accept': 'application/json',
                }
                plan_response = requests.get(full_plan_url, timeout=30, headers=plan_headers, allow_redirects=True)
                
                if plan_response.status_code != 200:
                    print(f"❌ Failed to download plan: {plan_response.status_code}")
                    continue
                
                plan = plan_response.json()
                print(f"✅ Plan downloaded: {len(plan.get('frames', []))} frames")
                
                # Update status to processing
                callback_headers = {
                    'User-Agent': 'Mozilla/5.0 (compatible; ColabBot/1.0)',
                    'Content-Type': 'application/json',
                }
                requests.post(callback_url, json={'status': 'processing', 'startedAt': True}, headers=callback_headers, timeout=10)
                
                # Save plan to file
                plan_file = f'{job_id}-plan.json'
                with open(plan_file, 'w') as f:
                    json.dump(plan, f)
                
                # Create render script (copies final MP4 to ./output/{job_id}.mp4)
                render_script = f'''import {{ renderStoryboardVideo }} from './server/services/remotion-ai-renderer';
import {{ readFileSync, mkdirSync, copyFileSync }} from 'fs';
import axios from 'axios';
import {{ join }} from 'path';

async function render() {{
  try {{
    const plan = JSON.parse(readFileSync('{plan_file}', 'utf-8'));
    console.log('Starting video render for job {job_id}...');
    const outputPath = await renderStoryboardVideo(plan);

    // Ensure deterministic output location for Colab download
    const outDir = './output';
    try {{ mkdirSync(outDir, {{ recursive: true }}); }} catch {{}}
    const canonicalOut = join(outDir, '{job_id}.mp4');
    try {{ copyFileSync(outputPath, canonicalOut); }} catch {{}}
    console.log('Done:', canonicalOut);

    await axios.post('{callback_url}', {{
      status: 'completed',
      outputPath: canonicalOut
    }});
  }} catch (e) {{
    const msg = e instanceof Error ? e.message : String(e);
    console.error('Render failed:', msg);
    await axios.post('{callback_url}', {{
      status: 'failed',
      error: msg
    }});
    process.exit(1);
  }}
}}

render();'''
                
                script_file = f'colab-render-{job_id}.ts'
                with open(script_file, 'w') as f:
                    f.write(render_script)
                
                # Run render
                print(f"🎥 Starting video render...")
                print(f"   Script: {script_file}")
                print(f"   This may take several minutes...")
                result = get_ipython().getoutput(f'npx --yes ts-node --transpile-only {script_file}')
                for line in result:
                    print(line)
                print(f"✅ Render completed for job {job_id}")
    else:
        print(f"⚠️  API returned status {response.status_code}")
        print("   Response:", response.text[:200])
        
except requests.exceptions.RequestException as e:
    print(f"❌ API connection failed: {e}")
    print(f"   Check that:")
    print(f"   1. Your API server is running")
    print(f"   2. ngrok tunnel is active")
    print(f"   3. API URL is correct: {api_url}")
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()
    print("")
    print("💡 Troubleshooting:")
    print(f"   1. Check API server: curl http://localhost:3000/health")
    print(f"   2. Check ngrok: ngrok http 3000")
    print(f"   3. Test URL: curl {api_url}/health")


## 7. Download Output Video


In [ ]:
# Find the output video file
import glob

output_files = glob.glob('output/*.mp4')
if output_files:
    latest_output = max(output_files, key=os.path.getctime)
    print(f"Found output video: {latest_output}")
    
    # Download the file
    files.download(latest_output)
    print("Download started!")
else:
    print("No output video found. Check the render logs above for errors.")
